# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../..")))

import sys
from pathlib import Path

# # Znajdź najbliższy katalog nadrzędny, który zawiera pakiet "RAG"
# p = Path.cwd().resolve()
# while True:
#     if (p / "RAG").is_dir():
#         sys.path.insert(0, str(p))
#         break
#     if p.parent == p:
#         raise RuntimeError("Nie znalazłem katalogu z folderem 'RAG' powyżej bieżącego notatnika.")
#     p = p.parent


from intergrax.supervisor.supervisor import IntergraxSupervisor, SupervisorConfig, SupervisorPromptPack
from intergrax.llm.llm_adapters import LLMAdapterRegistry
from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from langchain_ollama import ChatOllama
from components.compliance_checker import compliance_checker
from components.cost_estimator import cost_estimator
from components.final_summary import final_summary
from applications.UXAuditAgent.components.general_knowledge import general_intergrax_knowledge
from components.project_manager import project_manager
from components.ux_audit import ux_audit


# --- LLM adapter (Ollama) ---
adapter = LLMAdapterRegistry.create(
    name="ollama",
    chat=ChatOllama(model="llama3.1:latest"),
)

# --- Zasoby (opcjonalnie) ---
resources = {
    # "retriever": intergraxRagRetriever(...),
    # "ranker": intergraxReRanker(...),
}

# --- Rejestr komponentów ---
components = [
    general_intergrax_knowledge,
    ux_audit,
    final_summary,
    project_manager,
    final_summary,
    compliance_checker,
    cost_estimator,
]

embed_manager = IntergraxEmbeddingManager(
    verbose=True,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest", 
    assume_ollama_dim=1536)

# --- Supervisor z fallback + finally ---
cfg = SupervisorConfig(
    llm_adapter=adapter,
    components=components,
    heuristic_enable=False,
    debug=True,
    pass_temperature=False,
    pass_max_tokens=False,
    resources=resources,

    # runtime fallback only when a step has no component:
    fallback_component="Generalny",
    fallback_on_miss=True,

    # two-stage planner (decompose -> per-step assign)
    planner_mode="two_stage",
    plan_retries=2,
    assign_retries=2,
    assign_self_consistency=3,
    assign_threshold=0.50,          # zacznij luźniej, potem 0.6-0.7
    skip_on_low_confidence=False,   # zobacz przypisania nawet przy niskiej ufności
)

# --- Zapytanie testowe (jeden string) ---
query = (
    "Wykonaj następujące czynności wg podanej kolejności i warunków: "
    "1. Na podstawie przekazanych makiet FIGMY wciel się w rolę UX Audytora, "
    "który analizuje makiety pod kątem poprawności w logice i działaniu zaprojektowanej aplikacji. "
    "2. Sprawdź czy lista zmian jest zgodna z polityką prywatności firmy i z regulaminami. "
    "3. Przygotuj raport podsumowujący i przedstaw go Project Managerowi. "
    "4. Jeżeli PM odrzuci projekt — zrób krótkie podsumowanie i zakończ. Jeżeli go zaakceptuje, przejdź dalej. "
    "5. Wyciągnij raport finansowy za zeszły kwartał i oceń, czy budżet pozwala na te zmiany; "
    "jeśli koszt zmian < 2 procent budżetu — zatwierdź. "
    "6. Na koniec przygotuj pełny raport ze wszystkich kroków, działań i decyzji."
)

supervisor = IntergraxSupervisor(cfg)
plan = supervisor.plan(query)
supervisor.print_plan(plan)
state = supervisor.execute_plan(plan=plan, query=query, verbose=True)

c:\ProgramData\anaconda3\envs\genai_longchain\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



INTENT: procedural | Needs: rag=True, tools=True, general=True | Confidence: 1.00
1. Wcielić się w rolę UX Audytora
   Goal: Wcielić się w rolę UX Audytora
   How to execute: method=TOOL, depends_on=[]
   Component: Audytor UX
   Component found: True
   Assign score: 1.00 | low_confidence=False
   Inputs: ['przekazane makety FIGMY']
   Outputs: ['context_chunks', 'citations']
   Success criteria: ['wartościowe wyniki analizy']
   Fallback: Powtórzyć analizę
   Rationale:
     why_method: Domain operation handled by a specialized tool.
     why_component: Chosen by planner: Audytor UX
2. Sprawdzenie listy zmian
   Goal: Sprawdzenie listy zmian
   How to execute: method=RAG, depends_on=['S1']
   Component: Weryfikacja regulaminów i polityk prywatności
   Component found: True
   Assign score: 1.00 | low_confidence=False
   Inputs: ['lista zmian', 'polityka prywatności firmy', 'regulamin']
   Outputs: ['context_chunks', 'citations']
   Success criteria: ['wartościowe wyniki sprawdzenia'

In [ ]:
cfg = SupervisorConfig(
    llm_adapter=adapter,
    components=components,
    heuristic_enable=False,
    debug=True,
    pass_temperature=False,
    pass_max_tokens=False,
    resources={
        **resources,
        # plug-in your embedding function for semantic fallback:
        # it must be: callable(str) -> List[float]
        "embed_fn": resources.get("embed_fn"),  # np. embed_manager.embed_text
    },
    fallback_component="Generalny",
    fallback_on_miss=True,

    # Two-stage with stronger assignment
    planner_mode="two_stage",
    plan_retries=2,
    assign_retries=2,
    assign_self_consistency=3,
    assign_threshold=0.50,
    skip_on_low_confidence=False,
    assign_temperature=0.3,      # ważne: unikniesz „pustych” JSON-ów

    # Semantic router
    semantic_fallback=True,
    semantic_weight=0.15,
)

supervisor = IntergraxSupervisor(cfg)
plan = supervisor.plan(query)
supervisor.print_plan(plan)
state = supervisor.execute_plan(plan=plan, query=query, verbose=True)
print("KONIEC")



INTENT: procedural | Needs: rag=False, tools=True, general=True | Confidence: 0.90
1. Analiza makiety pod kątem poprawności logicznej i działania aplikacji
   Goal: Analiza makiety pod kątem poprawności logicznej i działania aplikacji
   How to execute: method=TOOL, depends_on=[]
   Component: Audytor UX
   Component found: True
   Assign score: 0.80 | low_confidence=False
   Inputs: ['makiety FIGMY']
   Outputs: ['context_chunks', 'citations']
   Success criteria: ['Poprawna analiza makiety']
   Fallback: Przeprowadzenie dodatkowej analizy
   Rationale:
     why_method: Domain operation handled by a specialized tool.
     why_component: Chosen by planner: Audytor UX
2. Sprawdzenie listy zmian
   Goal: Sprawdzenie listy zmian
   How to execute: method=TOOL, depends_on=['S1']
   Component: Weryfikacja regulaminów i polityk prywatności
   Component found: True
   Assign score: 1.00 | low_confidence=False
   Inputs: ['lista zmian', 'polityka prywatności firmy', 'regulaminów']
   Outputs: 